In [1]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv

#scientific computing
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats

import nibabel as nib

Makes design files for the ser, loc, and sim experiments

In [2]:
#preliminary stuff
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'scripts/sub_cb_mappings.txt'
os.chdir(home_dir)

event_dur = 0 #duration for all events

#get subject list
sub_list = []
sub_to_cb = {}
with open(subj_file) as f:
    for line in f:
        (key,val) = line.strip().split(' ') 
        sub_list.append('fd_' + key)
        sub_to_cb['fd_' + key] = int(val)

LOCALIZER

In [3]:
##get timing file for object localizer
def load_timing_file(run,time_to_drop):
    loc_file = home_dir + '/loc_timing/script_kidLoc_2Hz_run' + str(run)
    trial_type = []
    time = []
    with open(loc_file) as f:
        for n,line in enumerate(f):
            l = line.strip('\n').split('\t')
            if len(l)==5: #ignore file header and footer
                trial_type.append(int(l[2]))
                time.append(float(l[1]) - time_to_drop)
    return trial_type, time

In [4]:
#trial type dictionary for object localizer
# def trial_type_dict():
#     return {0:'scramble', 1:'face-adult', 2:'face-child', 3:'body', 4:'limb', 5:'car',
#             6:'guitar', 7:'place', 8:'house', 9:'word', 10:'number'}
def trial_type_dict():
    return {0:'scramble', 1:'face', 2:'face', 3:'body', 4:'body', 5:'object',
            6:'object', 7:'place', 8:'place', 9:'character', 10:'character'}

In [5]:
def write_loc_file(design_dir,cb):
    
    time_to_drop = -3
    trial_dict = trial_type_dict()
    duration = 4 #all blocks are 4s
    
    #set up run counterbalance
    if cb==1:
        run_dict = {1: '1', 2:'2'}
    else:
        run_dict = {1: '2', 2:'1'}
    
    #prepare out file
    out_file = design_dir + 'loc.csv'
    if os.path.exists(out_file):
        os.remove(out_file)
    out_f = open(out_file,'w')
    out_f.write('run,condition,onset,duration\n')
    for run in range(1,3):
        trial_type, time = load_timing_file(run,time_to_drop)

        #write out file
        for n,trial in enumerate(trial_type):
            if n > 0 and trial != trial_type[n-1]: #start of a new trial
                if trial_dict[trial] != 'scramble': #not a condition
                    new_line = ','.join([run_dict[run],trial_dict[trial],str(time[n]),str(4)+'\n'])
                    out_f.write(new_line)
    out_f.close()
    return

In [6]:
#write output
for sub in sub_list:
    design_dir = home_dir + 'data/' + sub + '/design/' #make design folder
    if not os.path.exists(design_dir):
        os.makedirs(design_dir)
    
    #write file
    write_loc_file(design_dir, sub_to_cb[sub])

Genereic code for both the SIM and SER experiments (processing log files, etc)

In [7]:
def process_log_files(log_files):
    full_data = {}
    for n,f in enumerate(log_files):
        
        #read file
        subj_file = open(os.path.join(data_dir,f),'r')
        subj_file = subj_file.readlines()
        subj_file = np.array(subj_file)
        
        ##process file into dictionary of dictionaries
        subj_dict = {}
        for n,line in enumerate(subj_file):
            subject_line = line.split(' ')[1].split('=')[0].split('%20') #0th index is subid, 1st is that lines id
            if len(subject_line) == 2:
                try: 
                    full_data[subject_line[0]][subject_line[1]] = line.split(' ')[1].split('=')[1].strip('\n')
                except:
                    full_data[subject_line[0]] = {} ##make dictionary
                    full_data[subject_line[0]][subject_line[1]] = line.split(' ')[1].split('=')[1].strip('\n')
    return full_data

In [14]:
##make subject lists
data_dir = '/home/iballard/Dropbox/fd/'

##process data for both sim and ser experiments
log_files = glob.glob(data_dir + 'data/scanning/*log')

##process log files
full_data = process_log_files(log_files)
temp_dict = {}
for f in full_data:
    temp_dict[f] = full_data[f]
full_data = temp_dict

In [9]:
#processes timing files to return times where a condition is rewarded, or not
def get_times(all_times,order,rew,cond_code,rew_code):
    cond = np.where([order == cond_code])[1]
    rewarded = np.where(rew[cond] == rew_code)
    indices = cond[rewarded]
    times = all_times[indices]
    return times

In [10]:
#parses the full data file to make data of interest more accesible
def load_timing_data(experiment,data):
    im_time = np.array(data['im_time_' + experiment].split(',')).astype('float')
    fb_time = np.array(data['fb_time_' + experiment].split(',')).astype('float')
    iti_time = np.array(data['iti_time_' + experiment].split(',')).astype('float')
    cond = np.array( map(float, data['cond_' + experiment].split(',') ))
    rew =  np.array( map(float, data['rew_' + experiment].split(',') ))
    order = np.array( map(float, data['trial_order_' + experiment].split(',')))
    
    #add iti times to im_time because cond indexing includes itis
    im_time = np.concatenate((iti_time,im_time),axis=0)
    im_time = np.sort(im_time)
        
    return im_time, fb_time, iti_time, cond, rew, order

SIM

In [11]:
#given info about run, event time,name, and duration, writes to lyman file
def write_entry(entry,time_to_drop,out_f,run,cond_name,event_dur,mvpa,scan_time):
    if not mvpa or (entry + 7)<scan_time: #only write if event is within scan time
        entry = entry - time_to_drop #adjust for dropped scans
        new_line = ','.join([str(run),cond_name,str(entry),str(event_dur)+'\n'])
        out_f.write(new_line)
    return out_f

In [21]:
def write_sim_file(sub,event_dur,mvpa):
    time_to_drop = 9.0 #number of seconds of scans to drop
    design_dir = home_dir + 'data/' + sub + '/design/'

    if mvpa:
        cond_dict = {1:'pair',2:'single',3:'pair',4:'single'}
        out_file = design_dir + 'sim_state.csv'
    else:
        cond_dict = {1:'AB_plus',2:'B_minus',3:'AC_minus',4:'C_plus'}
        out_file = design_dir + 'sim_noswitch.csv'

    #prepare out file
    if os.path.exists(out_file):
        os.remove(out_file)
    out_f = open(out_file,'w')
    out_f.write('run,condition,onset,duration\n')

    #add timing info for each of the events of interest
    for run in range(1,4):
        if sub == 'fd_126': #made a mistake in these sub #s. Check notes
            f = '125_' + str(run)
        elif sub == 'fd_127':
            f = '126_' + str(run)
        else:
            f = sub[-3:] + '_' + str(run) #get subject and run id
        
        
        out_run = run
        if sub == 'fd_127' and run == 1:
            continue #no run 1
        if sub == 'fd_127' and run == 2:
            out_run = 1
        if sub == 'fd_127' and run == 3:
            out_run = 2
        if sub == 'fd_133' and run == 2:
            continue #no run 2 of sim
        if sub == 'fd_133' and run == 3:
            out_run = 2
        
        #get length of run for that subject. For MVPA, want to make sure timing files are
        #cropped for runs that have been cropped
        crop_run = abspath('data/' + sub + '/func/sim/run_' + str(run) + '_fc_crop.nii.gz')
        if os.path.exists(crop_run):
            img = nib.load(crop_run)
        else:
            img = nib.load(abspath('data/' + sub + '/func/sim/run_' + str(run) + '_fc.nii.gz'))

        scan_time = img.shape[3]*1.5

        #load subject info
        im_time, fb_time, iti_time, cond, rew, order = load_timing_data('sim',full_data[f])

        #write condition files
        for condition in cond_dict.keys(): #loop through conditions
            for rew_or_switch in range(0,2): #loop through reward
                times = get_times(im_time,order,rew,condition,rew_or_switch)
                for entry in times:
                    cond_name = cond_dict[condition]
#                     if (rew_or_switch == 1 and condition in [2,3]) or \
#                         (rew_or_switch ==0 and condition in [1,4]): #surprise no reward
#                             cond_name = cond_name + '_switch'
                    out_f = write_entry(entry,time_to_drop,out_f,out_run,cond_name,event_dur,mvpa, scan_time)
   
        #add entry for feedback times (if not mvpa)
        if not mvpa:
            for entry in fb_time:
                out_f = write_entry(entry,time_to_drop,out_f,out_run,'feedback',event_dur,mvpa,scan_time)
                
    out_f.close()      

In [22]:
#write output
for sub in sub_list:
    write_sim_file(sub,event_dur,False)

SER

In [23]:
def write_ser_file(sub,event_dur,mvpa):
    time_to_drop = 9.0 #number of seconds of scans to drop
    condition = 'ser'
    design_dir = home_dir + 'data/' + sub + '/design/'

    if mvpa:
        cond_dict = {1:'A',2:'serial',3:'single',4:'A',5:'serial',6:'single'}
        out_file = design_dir + condition +'_state.csv'
    else:
        cond_dict = {1:'A',2:'B_plus',3:'B_minus',4:'A',5:'C_minus',6:'C_plus'}
        out_file = design_dir + condition +'_noswitch.csv'

    #prepare out file
    if os.path.exists(out_file):
        os.remove(out_file)
    out_f = open(out_file,'w')
    out_f.write('run,condition,onset,duration\n')

    #add timing info for each of the events of interest
    for run in range(1,4):
        if sub == 'fd_126': #made a mistake in these sub #s. Check notes
            f = '125_' + str(run)
        elif sub == 'fd_127':
            f = '126_' + str(run)
        else:
            f = sub[-3:] + '_' + str(run) #get subject and run id 
        
        #some obnoxious stuff to fix run ordering when there are missing runs
        out_run = run
        if sub == 'fd_127' and run == 1:
            continue #no run 1
        if sub == 'fd_127' and run == 2:
            out_run = 1
        if sub == 'fd_127' and run == 3:
            out_run = 2
        
        #get length of run for that subject. For MVPA, want to make sure timing files are
        #cropped for runs that have been cropped
        crop_run = abspath('data/' + sub + '/func/ser/run_' + str(run) + '_fc_crop.nii.gz')
        if os.path.exists(crop_run):
            img = nib.load(crop_run)
        else:
            img = nib.load(abspath('data/' + sub + '/func/ser/run_' + str(run) + '_fc.nii.gz'))

        scan_time = img.shape[3]*1.5
        
        #load subject info
        im_time, fb_time, iti_time, cond, rew, order = load_timing_data('ser',full_data[f])

        for condition in cond_dict.keys(): #loop through conditions
            if cond_dict[condition] != 'A': #A is never rewarded
                for rew_or_switch in range(0,2): #loop through reward
                    times = get_times(im_time,order,rew,condition,rew_or_switch)

                    for entry in times:
                        cond_name = cond_dict[condition]
#                         if (rew_or_switch == 1 and condition in [3,5]) or \
#                             (rew_or_switch ==0 and condition in [2,6]): #surprise no reward
#                                 cond_name = cond_name + '_switch'
                        out_f = write_entry(entry,time_to_drop,out_f,out_run,cond_name,event_dur, mvpa, scan_time)
            
            else: #A condition
                if not mvpa:
                    times = get_times(im_time,order,rew,condition,0)
                    for entry in times:
                        cond_name = cond_dict[condition]
                        out_f = write_entry(entry,time_to_drop,out_f,out_run,cond_name,event_dur, mvpa, scan_time)
                    
        #add entry for feedback times
        if not mvpa:
            for entry in fb_time:
                out_f = write_entry(entry,time_to_drop,out_f,out_run,'feedback',event_dur,mvpa, scan_time)      
    out_f.close() 

In [25]:
#write output
for sub in sub_list:
    write_ser_file(sub,event_dur,False)